**INITIALIZATION:**

In [10]:
#@ INITIALIZING NECESSARY PACKAGES AND DEPENDENCIES: 
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import datasets

from keras.models import Sequential 
from keras.layers.core import Dense
from tensorflow.keras.optimizers import SGD

import matplotlib.pyplot as plt
import numpy as np

**PERCEPTRON:**

In [2]:
#@ INITIALIZING PERCEPTRON MODEL: 
class Perceptron:                                       # Defining Perceptron. 
    def __init__(self, N, alpha=0.1):                   # Initializing Constructor Function. 
        self.W = np.random.randn(N + 1) / np.sqrt(N)    # Initializing Scaled Weight Matrix. 
        self.alpha = alpha                              # Initializing LR.
    
    def step(self, x):                                  # Defining Step Function. 
        return 1 if x > 0 else 0                        # Getting 1 if Positive else Negative. 
    
    def fit(self, X, y, epochs=10):                     # Defining Fit Function. 
        X = np.c_[X, np.ones((X.shape[0]))]             # Adding Column of Ones. 
        for epoch in np.arange(0, epochs):
            for (x, target) in zip(X, y):
                p = self.step(np.dot(x, self.W))        # Initializing Dot Product. 
                if p != target:
                    error = p - target                  # Computing Error. 
                    self.W += -self.alpha*error*x       # Updating Weight Matrix. 
    
    def predict(self, X, addBias=True):                 # Defining Predict Function. 
        X = np.atleast_2d(X)                            # Inspecting 2D Matrix. 
        if addBias:
            X = np.c_[X, np.ones((X.shape[0]))]         # Adding Bias Vector. 
        return self.step(np.dot(X, self.W))             # Initializing Dot Product. 

In [3]:
#@ EVALUATING PERCEPTRON BITWISE DATASETS: OR:
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])          # Initializing Array Example. 
y = np.array([[0], [1], [1], [1]])                      # Initializing Array Example. 

#@ TRAINING PERCEPTRON MODEL: 
p = Perceptron(X.shape[1], alpha=0.1)                   # Initializing Perceptron Model. 
p.fit(X, y, epochs=20)                                  # Training Model. 

#@ MODEL EVALUATION: 
for (x, target) in zip(X, y):
    pred = p.predict(x)
    print(f"data={x}, ground-truth={target}, pred={pred}")

data=[0 0], ground-truth=[0], pred=0
data=[0 1], ground-truth=[1], pred=1
data=[1 0], ground-truth=[1], pred=1
data=[1 1], ground-truth=[1], pred=1


**NEURAL NETWORKS: BACKPROPAGATION**

In [4]:
#@ INITIALIZING BACKPROPAGATION ALGORITHM:
class NeuralNetwork:                                            # Defining Neural Network. 
    def __init__(self, layers, alpha=0.1):                      # Initializing Constructor Function. 
        self.W = []                                             # Initialization. 
        self.layers = layers                                    # Initialization. 
        self.alpha = alpha                                      # Initialization. 
        for i in np.arange(0, len(layers) - 2):
            w = np.random.randn(layers[i]+1, layers[i+1]+1)     # Initializing Weight Matrix. 
            self.W.append(w / np.sqrt(layers[i]))               # Normalizing Variance. 
        w = np.random.randn(layers[-2]+1, layers[-1])           # Initializing Weight Matrix. 
        self.W.append(w / np.sqrt(layers[-2]))                  # Normalizing Variance. 
    
    def __repr__(self):                                         # Function for Debugging. 
        return "NeuralNetwork: {}".format(
            "-".join(str(l) for l in self.layers))              # Inspecting. 
    
    def sigmoid(self, x):                                       # Defining Sigmoid Function. 
        return 1.0 / (1 + np.exp(-x))                           # Implementation of Sigmoid Function. 
    
    def sigmoid_deriv(self, x):                                 # Defining Function. 
        return x * (1 - x)                                      # Getting Derivative of Sigmoid. 
    
    def fit(self, X, y, epochs=1000, displayUpdate=100):        # Defining Fit Function. 
        X = np.c_[X, np.ones((X.shape[0]))]                     # Adding Column of Ones. 
        for epoch in np.arange(0, epochs):
            for (x, target) in zip(X, y):
                self.fit_partial(x, target)                     # Backpropagation and Updating Matrix. 
            if epoch==0 or (epoch+1)%displayUpdate==0:
                loss = self.calculate_loss(X, y)                # Initializing Loss Calculation. 
                print("epoch={}, loss={:.7f}".format(
                    epoch+1, loss))                             # Inspecting Loss. 
    
    def fit_partial(self, x, y):                                # Defining Fit Partial Method. 
        A = [np.atleast_2d(x)]                                  # Initializing List. 

        #@ FEEDFORWARD:
        for layer in np.arange(0, len(self.W)):
            net = A[layer].dot(self.W[layer])                   # Dot Product of Activation and Weight Matrix. 
            out = self.sigmoid(net)                             # Implementing Nonlinear Activation Function. 
            A.append(out)                                       # Adding to Activations. 

        #@ BACKPROPAGATION:
        error = A[-1] - y                                       # Computing Error. 
        D = [error * self.sigmoid_deriv(A[-1])]                 # Initializing List of Deltas. 
        for layer in np.arange(len(A) - 2, 0, -1):
            delta = D[-1].dot(self.W[layer].T)                  # Initializing Dot Product. 
            delta = delta * self.sigmoid_deriv(A[layer])        # Implementating Derivative Function. 
            D.append(delta)                                     # Adding Activations. 
        D = D[::-1]                                             # Reversing Activations. 

        #@ WEIGHT UPDATE PHASE: 
        for layer in np.arange(0, len(self.W)):
            self.W[layer] += -self.alpha * A[layer].T.dot(D[layer])
        
    def predict(self, X, addBias=True):                         # Defining Predict Function. 
        p = np.atleast_2d(X)                                    # Initialization. 
        if addBias:
            p = np.c_[p, np.ones((p.shape[0]))]                 # Adding Column of Ones. 
        for layer in np.arange(0, len(self.W)):
            p = self.sigmoid(np.dot(p, self.W[layer]))          # Implementing Dot Product. 
        return p
    
    def calculate_loss(self, X, targets):                       # Defining Function. 
        targets = np.atleast_2d(targets)                        # Initialization. 
        predictions = self.predict(X, addBias=False)            # Implementation of Predict Method. 
        loss = 0.5 * np.sum((predictions - targets)**2)         # Computing Loss. 
        return loss

In [5]:
#@ EVALUATING NEURAL NETWORKS ON BITWISE DATASETS: XOR:
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])                  # Initializing Array Example. 
y = np.array([[0], [1], [1], [0]])                              # Initializing Array Example. 

#@ TRAINING NEURAL NETWORKS MODEL: 
nn = NeuralNetwork([2, 2, 1], alpha=0.5)                        # Initializing Neural Network.  
nn.fit(X, y, epochs=200)                                        # Training Model. 

#@ MODEL EVALUATION: 
for (x, target) in zip(X, y):
    pred = nn.predict(x)[0][0]
    step = 1 if pred > 0.5 else 0
    print(f"data={x}, ground-truth={target[0]}, pred={pred},"
          f"step={step}")

epoch=1, loss=0.5003620
epoch=100, loss=0.4991073
epoch=200, loss=0.4973769
data=[0 0], ground-truth=0, pred=0.4975027342572825,step=0
data=[0 1], ground-truth=1, pred=0.5119170447281081,step=1
data=[1 0], ground-truth=1, pred=0.503353226845293,step=1
data=[1 1], ground-truth=0, pred=0.5122126284343791,step=1


**MNIST SAMPLE:**

In [6]:
#@ GETTING THE DATASET:
digits = datasets.load_digits()                         # Initializing Dataset. 
data = digits.data.astype("float")                      # Converting into Float. 
data = (data - data.min()) / (data.max() - data.min())  # Normalizing Dataset. 
print("samples: {}, dim: {}".format(data.shape[0],
                                  data.shape[1]))       # Inspecting Dataset. 

samples: 1797, dim: 64


In [7]:
#@ PREPARING THE DATASET:
(train_X, test_X, train_Y, test_Y) = train_test_split(
    data, digits.target, test_size=0.25)                    # Splitting Dataset. 
train_Y = LabelBinarizer().fit_transform(train_Y)           # Converting Labels to Vectors. 
test_Y = LabelBinarizer().fit_transform(test_Y)             # Converting Labels to Vectors. 

#@ TRAINING THE NEURAL NETWORK:
nn = NeuralNetwork([train_X.shape[1], 32, 16, 10])          # Initializing Neural Network. 
nn.fit(train_X, train_Y, epochs=1000)                       # Training Neural Network.
 
#@ EVALUATING THE NEURAL NETWORK: 
predictions = nn.predict(test_X)                            # Getting Predictions. 
predictions = predictions.argmax(axis=1)                    # Getting Maximum Probability. 
print(classification_report(
    test_Y.argmax(axis=1), predictions))                    # Inspecting Classification Report. 

epoch=1, loss=605.7806718
epoch=100, loss=5.7756296
epoch=200, loss=2.9896694
epoch=300, loss=2.5347345
epoch=400, loss=2.3556062
epoch=500, loss=2.0657081
epoch=600, loss=1.2556464
epoch=700, loss=1.1949743
epoch=800, loss=1.1591630
epoch=900, loss=1.1346962
epoch=1000, loss=1.1167485
              precision    recall  f1-score   support

           0       0.98      0.94      0.96        52
           1       0.96      1.00      0.98        48
           2       1.00      0.95      0.97        40
           3       0.98      0.93      0.96        46
           4       0.95      1.00      0.97        38
           5       0.97      0.88      0.93        42
           6       0.96      1.00      0.98        52
           7       0.96      0.96      0.96        45
           8       0.97      0.97      0.97        40
           9       0.88      0.96      0.92        47

    accuracy                           0.96       450
   macro avg       0.96      0.96      0.96       450
weighted 

**NEURAL NETWORKS:**

In [16]:
#@ GETTING THE DATASET: 
dataset = datasets.fetch_openml("mnist_784")                # Downloading Dataset. 
data = dataset.data.astype("float") / 255.0                 # Initializing Data Normalization. 

#@ PREPARING THE DATASET:
(train_X, test_X, train_Y, test_Y) = train_test_split(
    data, dataset.target, test_size=0.25)                   # Splitting Dataset. 
train_Y = LabelBinarizer().fit_transform(train_Y)           # Converting Labels to Vectors. 
test_Y = LabelBinarizer().fit_transform(test_Y)             # Converting Labels to Vectors. 

In [17]:
#@ DEFINING NETWORK ARCHITECTURE: 
model = Sequential()                                            # Initializing Sequential Model. 
model.add(Dense(256, input_shape=(784,), activation="sigmoid")) # Initializing Dense Layer. 
model.add(Dense(128, activation="sigmoid"))                     # Adding Dense Layer. 
model.add(Dense(10, activation="softmax"))                      # Adding Dense Output Layer. 

#@ TRAINING NEURAL NETWORK:
sgd = SGD(0.01)                                                 # Initializing SGD Optimizer. 
model.compile(loss="categorical_crossentropy", optimizer=sgd, 
              metrics=["accuracy"])                             # Compiling Model. 
H = model.fit(train_X, train_Y,validation_data=(test_X,test_Y), 
              epochs=100, batch_size=128)                       # Training Model. 

Epoch 1/100
411/411 [==============================] - 5s 9ms/step - loss: 2.2734 - accuracy: 0.2027 - val_loss: 2.2424 - val_accuracy: 0.3757
Epoch 2/100
411/411 [==============================] - 4s 9ms/step - loss: 2.2117 - accuracy: 0.3997 - val_loss: 2.1809 - val_accuracy: 0.5271
Epoch 3/100
411/411 [==============================] - 3s 8ms/step - loss: 2.1409 - accuracy: 0.5214 - val_loss: 2.0989 - val_accuracy: 0.5877
Epoch 4/100
411/411 [==============================] - 3s 7ms/step - loss: 2.0442 - accuracy: 0.5869 - val_loss: 1.9851 - val_accuracy: 0.6066
Epoch 5/100
411/411 [==============================] - 3s 8ms/step - loss: 1.9111 - accuracy: 0.6264 - val_loss: 1.8310 - val_accuracy: 0.6790
Epoch 6/100
411/411 [==============================] - 3s 8ms/step - loss: 1.7409 - accuracy: 0.6637 - val_loss: 1.6460 - val_accuracy: 0.6683
Epoch 7/100
411/411 [==============================] - 3s 8ms/step - loss: 1.5495 - accuracy: 0.6915 - val_loss: 1.4522 - val_accuracy: 0.7088